In [56]:
%reload_ext autoreload
%autoreload 2

from sklearn.naive_bayes import ComplementNB, CategoricalNB, MultinomialNB, BernoulliNB, GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
import pandas as pd
from main import *

In [59]:
trial_time_spans = {
    "1 second": [7, 8],
    "2 seconds": [7, 9],
    "4 seconds": [7, 11],
    "6 seconds": [7, 13],
    "8 seconds": [7, 15],
    "10 seconds": [5, 15],
    "12 seconds": [3, 15],
    "15 seconds": [0, 15],
}

classifiers = {
    # 'ComplementNB': ComplementNB(),
    # 'CategoricalNB': CategoricalNB(),
    # 'MultinomialNB': MultinomialNB(),
    # 'GaussianNB': GaussianNB(),

    'BernoulliNB': BernoulliNB(),
    'XGBClassifier': XGBClassifier(n_estimators=300, max_depth=6, learning_rate=0.1, objective='binary:logistic'),
    'AdaBoostClassifier': AdaBoostClassifier(algorithm="SAMME", learning_rate=0.1, n_estimators=200),
    'MLP': MLPClassifier(),
    'RF': RandomForestClassifier(n_estimators=300),
    'KNN': KNeighborsClassifier(n_neighbors=3),
    'LDA': LinearDiscriminantAnalysis(solver='svd'),
    'SVC_poly': SVC(kernel="poly", degree=3),
    'SVC_linear': SVC(kernel="linear"),
    'SVC_sigmoid': SVC(kernel="sigmoid"),
    'SVC_rbf': SVC(kernel="rbf"),
    'GradientBoostingClassifier': GradientBoostingClassifier(),
    'LGBMClassifier': LGBMClassifier(),
}

def get_data(trial_time_span):
    trials, y = get_trials_and_labels(trial_kwargs={'crop_start_end': trial_time_span})

    x = [[v for k, v in i.features.items() if 'time' in k] for i in trials]
    x = np.array(x)

    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    ros = RandomOverSampler(random_state=0)
    x, y = ros.fit_resample(x, y)

    x, y = shuffle(x, y, random_state=0) or exit(1)

    return x, y


def get_accuracies():
    accuracies = {k: [] for k in classifiers}

    for i in range(2):
        for classifier_name, classifier in classifiers.items():
            print(f"Classifier: {classifier_name}")

            cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=0)
            score = cross_val_score(classifier, x, y, scoring='accuracy', cv=cv, n_jobs=-1)
            accuracies[classifier_name].extend(score)

    print(*sorted([f"{k}, {round(np.mean(v), 3)}" for k, v in accuracies.items()]), sep="\n")

    # Total mean
    print("Total mean:", round(np.mean([np.mean(v) for v in accuracies.values()]), 3))

    return accuracies

In [60]:
duration_accs = []

for trial_time_span_name, trial_time_span in trial_time_spans.items():
    print(f"Trial time span: {trial_time_span_name}")

    x, y = get_data(trial_time_span)
    accuracies = get_accuracies()

    duration_accs.append([np.mean(v) for v in accuracies.values()])

dur_acc_df = pd.DataFrame(duration_accs, columns=classifiers.keys(), index=trial_time_spans.keys())

Trial time span: 1 second
Getting epochs for filename data/dasps_raw_mat/S01.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S01.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S02.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S02.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S02.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S02.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs

/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690:

Classifier: RF
Classifier: KNN
Classifier: LDA
Classifier: SVC_poly
Classifier: SVC_linear
Classifier: SVC_sigmoid
Classifier: SVC_rbf
Classifier: GradientBoostingClassifier
Classifier: LGBMClassifier
[LightGBM] [Info] Number of positive: 124, number of negative: 125
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013719 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13390
[LightGBM] [Info] Number of positive: 125, number of negative: 124
[LightGBM] [Info] Number of data points in the train set: 249, number of used features: 182
[LightGBM] [Info] Number of positive: 125, number of negative: 125
[LightGBM] [Info] Number of positive: 125, number of negative: 125
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.497992 -> initscore=-0.008032
[LightGBM] [Info] Start training from score -0.008032
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001876 seconds.
You 

/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690:

Classifier: RF
Classifier: KNN
Classifier: LDA
Classifier: SVC_poly
Classifier: SVC_linear
Classifier: SVC_sigmoid
Classifier: SVC_rbf
Classifier: GradientBoostingClassifier
Classifier: LGBMClassifier
[LightGBM] [Info] Number of positive: 125, number of negative: 124
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001696 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13522
[LightGBM] [Info] Number of data points in the train set: 249, number of used features: 182
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502008 -> initscore=0.008032
[LightGBM] [Info] Start training from score 0.008032
[LightGBM] [Info] Number of positive: 124, number of negative: 125
[LightGBM] [Info] Number of positive: 125, number of negative: 125
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000910 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Inf

...

Classifier: RF
Classifier: KNN
Classifier: LDA
Classifier: SVC_poly
Classifier: SVC_linear
Classifier: SVC_sigmoid
Classifier: SVC_rbf
Classifier: GradientBoostingClassifier
Classifier: LGBMClassifier
[LightGBM] [Info] Number of positive: 125, number of negative: 124
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000792 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13537
[LightGBM] [Info] Number of data points in the train set: 249, number of used features: 182
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502008 -> initscore=0.008032
[LightGBM] [Info] Start training from score 0.008032
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 124, number of negative: 125
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001160

/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690:

Classifier: RF
Classifier: KNN
Classifier: LDA
Classifier: SVC_poly
Classifier: SVC_linear
Classifier: SVC_sigmoid
Classifier: SVC_rbf
Classifier: GradientBoostingClassifier
Classifier: LGBMClassifier
[LightGBM] [Info] Number of positive: 125, number of negative: 124
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001190 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13537
[LightGBM] [Info] Number of data points in the train set: 249, number of used features: 182
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502008 -> initscore=0.008032
[LightGBM] [Info] Start training from score 0.008032
[LightGBM] [Info] Number of positive: 124, number of negative: 125
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001479 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13424
[LightGBM] [Info] Number of data points in the 

/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/jakubblaha/.local/share/virtualenvs/thesis-python-yCK4mToH/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690:

Classifier: RF
Classifier: KNN
Classifier: LDA
Classifier: SVC_poly
Classifier: SVC_linear
Classifier: SVC_sigmoid
Classifier: SVC_rbf
Classifier: GradientBoostingClassifier
Classifier: LGBMClassifier
[LightGBM] [Info] Number of positive: 125, number of negative: 124
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000727 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13535
[LightGBM] [Info] Number of data points in the train set: 249, number of used features: 182
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.502008 -> initscore=0.008032
[LightGBM] [Info] Start training from score 0.008032
[LightGBM] [Info] Number of positive: 124, number of negative: 125
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001016 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

In [ ]:
print(dur_acc_df.T)

In [ ]:
dur_acc_df.plot(kind='bar', figsize=(15, 10))
plt.show()

NameError: name 'dur_acc_df' is not defined

Getting epochs for filename data/dasps_raw_mat/S01.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S01.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S02.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S02.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S02.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_raw_mat/S02.mat
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
Getting epochs for filename data/dasps_r

KeyboardInterrupt: 